In [ ]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler, get_openai_callback
from langchain.llms.loading import load_llm

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
chat = load_llm("model.json")
chat

# gpt API usage
# with get_openai_callback() as usage:
#     a = chat.predict("What is the recipe for soju")
#     print(a, "\n")
#     print(usage)

In [ ]:
# Output Parser

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))

In [ ]:
# Template

template = ChatPromptTemplate.from_messages([
    ("system", "You are a rocket scientist working for SpaceX. You are a specialist in the field of Rocket Engineering. If I ask you questions about rocket science, even if it's extremely complicated and difficult to explain, you must answer all information. And if you don't exactly know the answer, you should say you don't know."),
    ("human", "{question}")
])

In [ ]:
# Chaining

chain = template | chat

# chain.invoke({"question": "Tell me about the overall rocket science including some formulas related to launching rockets."})

In [ ]:
# FewShotPromptTemplate

from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
        """,
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat
chain.invoke({"country": "South Korea"})

In [ ]:
# FewShotChatMessagePromptTemplate

from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

examples = [
    {
        "country": "France",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. You give short answers."),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat
chain.invoke({"country": "Germany"})

In [ ]:
# LengthBasedExampleSelector

from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {
        "question": "France",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
        """,
    },
    {
        "question": "Italy",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
        """,
    },
    {
        "question": "Greece",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

In [ ]:
# Cacheing

from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))

chat.predict("How can I start my own business?")

In [ ]:
response = chat.predict("How can I start my own business?")
response

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/1984.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever=vectorstore.as_retriever()

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)
chain.run("Describe Victory Mansions")

'Victory Mansions is a building where Winston Smith resides. The hallway of Victory Mansions has a smell of boiled cabbage and old rag mats. At one end of the hallway, there is a large colored poster of a man\'s face, about forty-five years old, with a heavy black mustache and ruggedly handsome features. The building has seven flights of stairs, as the lift is often not working and the electric current is cut off during daylight hours as part of an economy drive. On each landing, there is a poster with the caption "BIG BROTHER IS WATCHING YOU." The flat inside Victory Mansions is where Winston lives, and it is described as small and cramped. The flat has a telescreen, an oblong metal plaque that cannot be completely shut off, constantly broadcasting information.'

In [15]:
results = vectorstore.similarity_search("where does wiston live")
len(results)

4

In [12]:
# Embedding

from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_documents([
    "Hi",
    "how",
    "are",
    "you"
])

print(len(vector), len(vector[0]))

4 1536
